# Fine-tuning the modified model with our data

## Imports


In [3]:
# connecting drive to colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torchvision
import numpy as np
import torch
import argparse
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import torch.nn as nn
from collections import OrderedDict
import torch.optim as optim
import time
import copy
import torchvision.transforms as transforms
from torchvision.io.image import read_image
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader

ModuleNotFoundError: ignored

## Helper functions

In [ ]:
# converting all the images to tensors and then normalize them
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

## Our prediction model

In [ ]:
# Initialize model with the best available weights
# create an instance of (e.g.) torchvision.models.segmentation.fcn_resnet50
# and tell it to load pretrained weights
weights = FCN_ResNet50_Weights.DEFAULT
modified_model = fcn_resnet50(weights=weights)

# set computation device
device = torch.device('cuda')

# model to train() and load onto computation devicce
modified_model.train().to(device)

# we are feature extracting so we only need to compute weights for the new layer
set_parameter_requires_grad(modified_model, True)

In [ ]:
# modify that model by removing its final layer and replacing with a 2D vector output at each pixel(?) (instead of 20 class logits)

# (classifier): FCNHead(
#     (0): Conv2d(2048, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
#     (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#     (2): ReLU()
#     (3): Dropout(p=0.1, inplace=False)
#     (4): Conv2d(512, 21, kernel_size=(1, 1), stride=(1, 1))
#   )
# (aux_classifier): FCNHead(
#     (0): Conv2d(1024, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
#     (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#     (2): ReLU()
#     (3): Dropout(p=0.1, inplace=False)
#     (4): Conv2d(256, 21, kernel_size=(1, 1), stride=(1, 1))
#   )
# )

# instead of torch.Size([1, 21, 120, 240]) -> torch.Size([1, 2, 120, 240])
modified_model.classifier[3] = nn.Sequential()
modified_model.classifier[4] = nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))
modified_model.aux_classifier[3] = nn.Sequential()
modified_model.aux_classifier[4] = nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
print(modified_model)

In [ ]:
# see what parameters will be tuned
params_to_update = modified_model.parameters()
print('Params to learn:')
for name, param in modified_model.named_parameters():
  if param.requires_grad:
      print(name)

### Train function

In [ ]:
# define train function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_aux=False):
  since = time.time()

  val_acc_history = []

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0 # shall we have in our case?

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
              inputs = inputs.to(device)
              labels = labels.to(device)

              # zero the parameter gradients
              optimizer.zero_grad()

              # forward
              # track history if only in train
              with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_aux and phase == 'train': 
                      outputs, aux_outputs = model(inputs) # should we use aux? this is from inception model, aux works differently in fcn resnet
                      loss1 = criterion(outputs, labels)
                      loss2 = criterion(aux_outputs, labels)
                      loss = loss1 + 0.4*loss2
                    else:
                      outputs = model(inputs)
                      loss = criterion(outputs, labels)
                    
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

            print()
            
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best val Acc: {:4f}'.format(best_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
        return model, val_acc_history


## Loading our dataset

In [ ]:
TRAIN_DATA = "training_data_2023-01-02"
!unzip -d "$TRAIN_DATA"/ /content/drive/MyDrive/"$TRAIN_DATA".zip # unziping training data

In [7]:
import sys
sys.path.append('/content/drive/MyDrive/drive_folder')
from custom_dataset_loader import TaiChiDataset, ToTensor

# Load dataset
dataset = TaiChiDataset(log_file=TRAIN_DATA+'/sample_ids.txt',
                        root_dir=TRAIN_DATA,
                        check=True,
                        transform=ToTensor()
                        )

print(len(dataset)) 

# and load data
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

ModuleNotFoundError: ignored

## Training & evaluating the model

In [ ]:
batch_size = 8
num_epochs = 15

# define optimizer
my_optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# Setup the loss
my_criterion = nn.CrossEntropyLoss()

# Train and evaluate
modified_model, hist = train_model(modified_model,
                                   dataloaders_dict,
                                   my_criterion,
                                   my_optimizer,
                                   num_epochs=num_epochs)